Hydrous DB importer / manager
Writen by LW/ES 20171224

Declarations and imports

In [2]:
import pandas
import os
import glob2
import json
import io
import piexif

declare globals and init variables

In [18]:
#Make sure to change root directory to the directory where your  image file structure starts - will add GUI in future but for now this is fastest. 

#--------------------------------CHANGE DIRECTORY BELOW------------------
rootdir = '/Users/lucas/Documents/hydrousdb/hyrdousdb/data'
#----------------------------------------------------------
#rootdir = 'C:\\Users\\lucasw\\Documents\\hyrdousdb\\data\\'
searchdir = rootdir + '/**/*'
twod_extention_list = ['.jpg','.png']
threed_extention_list = ['.obj', '.stl']


In [4]:
# static functions
os.getcwd() # just a quick reference to see what our current working directory (CWD) is. Doesn't need to be run.  

'C:\\Users\\lucasw\\Documents\\hyrdousdb'

declare classes - These are the objects that make up our database before converting to a pandas dataframe.

In [5]:
class commondata(object):
    def __iter__(self):
        for attr, value in self.__dict__.iteritems(): #only return indicies
            yield attr, value
    def __str__(self):
        return "This is data common to all data sources"
    def __init__(self):
        #input from text file, init to blank
        self.textfile_data = {'projname' : '',
                              'date' : '',
                              'location' : '',
                              'latitude' : '',
                              'longitude' : '',
                              'photographer' : '',
                              'species' : '',
                             }
        
        #passed from program
        self.file_info = {'filename' : '',
                          'file_extension' : '',
                          'fileroot' : '',
                          'size' : ''
                          }
        


    def importfromtext(self, import_textfile): 
        self.textfile_data = import_textfile.textfile_data

class info_text_file(commondata):
    def __str__(self):
        return "This is a metadata object, which inherets a lot of parameters from commondata class."
    def __init__(self, filename, fileroot):
        commondata.__init__(self)
        self.file_info['filename'] = filename
        self.file_info['fileroot'] = fileroot        
    def process(self):
        print( 'found json file ' + self.file_info['filename'] +', importing values')
        with open(self.file_info['filename'], 'r') as f:
            datastore = json.load(f)
        #this might be more effecient if we make a common function which iterates through keys
        for key in datastore:
            if key in self.textfile_data:
                self.textfile_data[key] = datastore[key]
            if key not in self.textfile_data:
                print( "Warning, extra data found in JSON, appending to new key " + key)
                self.textfile_data[key] = datastore[key]
            

      
class twod_img(commondata):
    def __str__(self):
        return "This is a 2d image object, which inherets a lot of parameters from commondata class."
    def __init__(self, filename, file_extension, fileroot):
        commondata.__init__(self)
        self.file_info['filename'] = filename
        self.file_info['fileroot'] = fileroot
        self.file_info['file_extension'] = file_extension
        self.file_info['size'] = os.path.getsize(self.file_info['filename'])
        self.supplemental_data = {'resolution' : '',
                          'colorcorrected_image' : False
                         }
        
        
    def extract_exif(self):
        exif_dict = piexif.load(self.file_info['filename'])
        #dump(exif_dict) #- Get exif as bytes to save with JPEG.
        #insert(exif_bytes, filename) #- Insert exif into JPEG.
        #remove(filename) #- Remove exif from JPEG.)

        #extract exif data from 2d image

class threed_model(commondata):
    def __str__(self):
        return "This is a 3d model object, which inherets a lot of parameters from commondata class."
    def __init__(self, filename, file_extension, fileroot):
        commondata.__init__(self)
        self.file_info['filename'] = filename
        self.file_info['fileroot'] = fileroot
        self.file_info['file_extension'] = file_extension
        self.file_info['size'] = os.path.getsize(self.file_info['filename'])
        self.supplemental_data = {}
        
    #def collect_files(self):
        #create associative link between texture, 3d, and other files of same model (.mtl, .obj, .jpg)

In [6]:
import_dict = {}
json_identifier_dict = {}

for f in glob2.iglob(searchdir + '.json'):
    fileroot = os.path.dirname(f)
    if fileroot in json_identifier_dict:
        print('WARNING: multiple text files found in folder, ignoring ' + f)
        pass
    if fileroot not in json_identifier_dict :    
        json_identifier_dict[f] = info_text_file(f, fileroot)
        json_identifier_dict[f].process() #process the metadata
#need to check for nested .json files to avoid problems.
#create copy of json identifier dicts so it can be modified - dicts are immutable
json_identifier_dict_cleaned = json_identifier_dict.copy()

for key1 in json_identifier_dict:
    for f in glob2.iglob(json_identifier_dict[key1].file_info['fileroot'] + '/*/**/*' + '.json'):
        json_identifier_dict_cleaned.pop(f)
        print('WARNING: json file found in subdirectory. Ignoring ' + f)
        pass
        
for key in json_identifier_dict_cleaned:
    for f in glob2.iglob(json_identifier_dict_cleaned[key].file_info['fileroot'] + '/**/*'):
        filename, file_extension = os.path.splitext(f)
        fileroot = os.path.dirname(f)
        if file_extension in ['.json']:
            print('WARNING: json file found in subdirectory. Ignoring ' + f)
            pass
        if file_extension in twod_extention_list:
            import_dict[f] = twod_img(f, file_extension, fileroot)
            import_dict[f].importfromtext(json_identifier_dict_cleaned[key])
            pass
        if file_extension in threed_extention_list:
            import_dict[f] = threed_model(f, file_extension, fileroot)
            import_dict[f].importfromtext(json_identifier_dict_cleaned[key])
            pass
        if file_extension not in threed_extention_list + twod_extention_list:
            print('WARNING: Filetype not recognized, ignoring ' + f)
            pass
        else:
            pass


C:\Users\lucasw\Documents\hyrdousdb\data\1-Model-Verticle wall_jpodgs\data copy.json
found json file C:\Users\lucasw\Documents\hyrdousdb\data\1-Model-Verticle wall_jpodgs\data copy.json, importing values
C:\Users\lucasw\Documents\hyrdousdb\data\1-Model-Verticle wall_jpodgs\data.json
found json file C:\Users\lucasw\Documents\hyrdousdb\data\1-Model-Verticle wall_jpodgs\data.json, importing values
C:\Users\lucasw\Documents\hyrdousdb\data\2-Model-Blue soft coral-jpgs\data.json
found json file C:\Users\lucasw\Documents\hyrdousdb\data\2-Model-Blue soft coral-jpgs\data.json, importing values


In [21]:
#This block makes a pandas database from a dictionary, then it exports it to the current working directory.

pandas_dict = []
for key in import_dict:   
    pandadict_element = {}
    pandadict_element.update(import_dict[key].file_info)
    pandadict_element.update(import_dict[key].textfile_data)
    pandadict_element.update(import_dict[key].supplemental_data)
    pandas_dict.append(pandadict_element)

pandas_DF = pandas.DataFrame.from_dict(pandas_dict)

#create a new DB with today's date and time - we can change this to update an existing DB in the future, but for now it's nice to have past versions for debug

writer = pandas.ExcelWriter(("Image_Database_{}.xlsx".format(pandas.datetime.today().strftime('%y%m%d-%H%M%S'))))
pandas_DF.to_excel(writer,'Image Database')
writer.save()
print('File Exported to: ' + os.getcwd())


File Exported to: C:\Users\lucasw\Documents\hyrdousdb


In [ ]:
for ifd_name in exif_dict:
    print("\n{0} IFD:".format(ifd_name))
    for key in exif_dict[ifd_name]:
        try:
            print(key, exif_dict[ifd_name][key][:10])
        except:
            print(key, exif_dict[ifd_name][key])

In [ ]:
Old / test functions - IGNORE





































In [7]:
fart = {}
for key in import_dict:
        attribute_index = [a for a in dir(import_dict[key]) if not a.startswith('__')]
        print attribute_index 
        fart.update(import_dict[key].file_info)
        fart.update(import_dict[key].textfile_data)
        print fart

['extract_exif', 'file_info', 'importfromtext', 'supplemental_data', 'textfile_data']
{'projname': u'maldives cruise', 'photographer': u'David Turtleman', 'file_extension': '.jpg', 'location': u'Pittsburgh', 'fileroot': 'C:\\Users\\lucasw\\Documents\\hyrdousdb\\data\\1-Model-Verticle wall_jpodgs\\raw', 'filename': 'C:\\Users\\lucasw\\Documents\\hyrdousdb\\data\\1-Model-Verticle wall_jpodgs\\raw\\20171211-_32B6235.jpg', 'date': u'20171227', 'longitude': 73.185691, 'latitude': 0.216724, 'species': u'Lame Coral', 'size': 14858279L}
['extract_exif', 'file_info', 'importfromtext', 'supplemental_data', 'textfile_data']
{'projname': u'maldives cruise', 'photographer': u'David Turtleman', 'file_extension': '.jpg', 'location': u'Pittsburgh', 'fileroot': 'C:\\Users\\lucasw\\Documents\\hyrdousdb\\data\\2-Model-Blue soft coral-jpgs', 'filename': 'C:\\Users\\lucasw\\Documents\\hyrdousdb\\data\\2-Model-Blue soft coral-jpgs\\20171211-_32B6326.jpg', 'date': u'20171227', 'longitude': 73.185691, 'latitu

testjson = {'projname': 'testfile',
           'date' : '20171227',
           'location' : 'Maldives',http://localhost:8888/notebooks/hyrdousdb/db_process.ipynb#
           'photographer' : 'Elle Stapleton',
           'species' : 'Cool Coral',
            'latitude' : 0.216724, 
            'longitude' : 73.185691
           }
json_string = json.dumps(testjson)
fart = pandas.Series(testjson)
fart
fart.to_json('data.json')

In [49]:
filepath = '/Users/lucas/Documents/hydrousdb/hyrdousdb/data/3-Model-Black coral-jpgs/20171211-_32B6368.jpg'
exif_dict = piexif.load(filepath)
for key in exif_dict:
    print key
print exif_dict

o = io.BytesIO()
thumb_im = Image.open("foo.jpg")
thumb_im.thumbnail((50, 50), Image.ANTIALIAS)
thumb_im.save(o, "jpeg")
thumbnail = o.getvalue()

zeroth_ifd = {piexif.ImageIFD.Make: u"Canon",
              piexif.ImageIFD.XResolution: (96, 1),
              piexif.ImageIFD.YResolution: (96, 1),
              piexif.ImageIFD.Software: u"piexif"
              }
exif_ifd = {piexif.ExifIFD.DateTimeOriginal: u"2099:09:29 10:10:10",
            piexif.ExifIFD.LensMake: u"LensMake",
            piexif.ExifIFD.Sharpness: 65535,
            piexif.ExifIFD.LensSpecification: ((1, 1), (1, 1), (1, 1), (1, 1)),
            }
gps_ifd = {piexif.GPSIFD.GPSVersionID: (2, 0, 0, 0),
           piexif.GPSIFD.GPSAltitudeRef: 1,
           piexif.GPSIFD.GPSDateStamp: u"1999:99:99 99:99:99",
           }
first_ifd = {piexif.ImageIFD.Make: u"Canon",
             piexif.ImageIFD.XResolution: (40, 1),
             piexif.ImageIFD.YResolution: (40, 1),
             piexif.ImageIFD.Software: u"piexif"
             }

exif_dict = {"0th":zeroth_ifd, "Exif":exif_ifd, "GPS":gps_ifd, "1st":first_ifd, "thumbnail":thumbnail}
exif_bytes = piexif.dump(exif_dict)
im = Image.open("foo.jpg")
im.thumbnail((100, 100), Image.ANTIALIAS)
im.save("out.jpg", exif=exif_bytes)

ValueError: Got invalid value.